In [14]:
import numpy as np
import pandas as pd
import random
from typing import List, Tuple, Dict
from itertools import combinations
from geopy.distance import geodesic
from collections import deque


POPULATION_SIZE = 300              # Population size
OFFSPRING_SIZE = 100                # Number of offspring per generation
MAX_GENERATIONS = 3000             # Maximum number of generations
MUTATION_RATE = 0.99               # Mutation probability
'''
# Loading cities and building the distance matrix
CITIES = pd.read_csv('./italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

'''
None



In [15]:
def load_multiple_countries(file_paths: Dict[str, str]) -> Dict[str, Tuple[pd.DataFrame, np.ndarray]]:
    data = {}
    for country, path in file_paths.items():
        cities = pd.read_csv(path, header=None, names=['name', 'lat', 'lon'])
        dist_matrix = np.zeros((len(cities), len(cities)))
        for c1, c2 in combinations(cities.itertuples(), 2):
            dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geodesic(
                (c1.lat, c1.lon), (c2.lat, c2.lon)
            ).km
        data[country] = (cities, dist_matrix)
    return data

In [18]:
file_paths = {
    'Italy': './italy.csv',
    'Russia': './russia.csv'
    
}
data = load_multiple_countries(file_paths)


## Fitness

In [19]:
# Function to calculate the path cost (total length)
def path_cost(country_name: str, path: List[int], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> float:
    cities, dist_matrix = data[country_name]
    num_cities = len(cities)
    return sum(dist_matrix[path[i], path[(i + 1) % num_cities]] for i in range(num_cities))
# Function to calculate the path cost (total length)
def path_cost_circular(country_name: str, path: List[int], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> float:
    total_cost = 0.0
    _, dist_matrix = data[country_name]
    for i in range(len(path)):
        total_cost += dist_matrix[path[i], path[(i + 1) % len(path)]]
    return total_cost

# Fitness function: inverse of the path length
def fitness(country_name: str, individual: List[int], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> float:
    return -path_cost(country_name, individual, data)

def fitness_circular(country_name: str, individual: List[int], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> float:
    return -path_cost_circular(country_name, individual, data)


## Inizialization

In [20]:
def KKN(country_name: str, start_city: str, data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> List[int]:
    cities, dist_matrix = data[country_name]
    start_city_index = cities[cities['name'] == start_city].index[0]
    path = [start_city_index]
    visited = set(path)
    current_city = start_city_index
    while len(visited) < len(cities):
        nearest_distance = float('inf')
        # Find the nearest cities
        for city_index in range(len(cities)):
            if city_index not in visited:
                distance = dist_matrix[current_city, city_index]
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_city = city_index
        path.append(nearest_city)
        visited.add(nearest_city)
        current_city = nearest_city
    path.append(start_city_index)# return to the starting point
    # Local optimization: 2-opt
    return tweak(country_name, path, data)


def tweak(country_name: str, path: List[int], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> List[int]:
    """ Performs 2-opt optimization on the given path. """
    improved = True
    while improved:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path)):
                if j - i == 1: # Avoid reversing two consecutive points
                    continue
                new_path = path[:]
                new_path[i:j] = reversed(path[i:j])  # Reverse the subsequence
                if path_cost(country_name, new_path, data) < path_cost(country_name, path, data):
                    path = new_path
                    improved = True
    return path


def initialize_population(country_name: str, data: Dict[str, Tuple[pd.DataFrame, np.ndarray]], size: int, start_city: str) -> List[List[int]]:

    cities, _ = data[country_name]
    num_cities = len(cities)
    # Find the index of the starting city
    start_city_index = cities[cities['name'] == start_city].index[0]
    # List of city indices excluding the starting one
    other_cities = [i for i in range(num_cities) if i != start_city_index]
    # Generate the population by fixing the starting and ending city
    population = []
    for _ in range(size):
        individual = [start_city_index]
        individual += list(np.random.permutation(other_cities)) # Permutation of other cities
        individual.append(start_city_index)
        population.append(individual)
    return population


def initialize_population_with_greedy_and_shuffle(country_name: str, data: Dict[str, Tuple[pd.DataFrame, np.ndarray]], size: int, start_city: str) -> List[List[int]]:

    
    cities, _ = data[country_name]
    # Find the index of the starting city
    start_city_index = cities[cities['name'] == start_city].index[0]
    
    
    # List of city indices excluding the starting one
    other_cities = [i for i in range(len(cities)) if i != start_city_index]
    
   
    population = []
    
    # Generate 5 greedy paths and add them to the population
    for _ in range(5):
        greedy_path = KKN(country_name, start_city, data)
        population.append(greedy_path)
    
    
    for _ in range(size - 5):
        individual = [start_city_index]
        individual += list(np.random.permutation(other_cities))
        individual.append(start_city_index)
        population.append(individual)
    
    
    random.shuffle(population)
    
    return population

def initialize_population_buffer(country_name: str, data: Dict[str, Tuple[pd.DataFrame, np.ndarray]], size: int) -> deque:
   # Initialize the circular buffer with a fixed maximum size
    population_buffer = deque(maxlen=size)
    
    
    cities, _ = data[country_name]
    # List of all city indices
    all_cities = list(range(len(cities)))
    
    # Generate the population by creating random permutations of all cities
    for _ in range(size):
        individual = list(np.random.permutation(all_cities))
        population_buffer.append(individual)
    
    return population_buffer


In [33]:
population_all = []
population_all_buffer = []
population_all_greeedy = []
for country, (cities, _) in data.items():
    population = initialize_population(country, data, POPULATION_SIZE, cities['name'][0])
    population_all.append(population)
    population_buffer = initialize_population_buffer(country, data, POPULATION_SIZE)
    population_all_buffer.append(population_buffer)
    population_greedy = initialize_population_with_greedy_and_shuffle(country, data, POPULATION_SIZE, cities['name'][0])
    population_all_greeedy.append(population_greedy)



## Parent selection function

In [21]:
# Parent selection using tournament
def parent_selection(country_name: str, population: List[List[int]], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> List[int]:
    tournament = random.sample(population, 5)
    tournament.sort(key=lambda ind: fitness(country_name, ind, data), reverse=True)
    return tournament[0]

def parent_selection_circular(country_name: str, population: List[List[int]], data: Dict[str, Tuple[pd.DataFrame, np.ndarray]]) -> List[int]:
    _, dist_matrix = data[country_name]
    tournament = random.sample(population, 5)
    tournament.sort(key=lambda ind: fitness_circular(country_name, ind, dist_matrix), reverse=True)
    return tournament[0]


## Crossover functions

In [22]:
def inver_over_crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    # Copy the first parent's path to start creating the child
    
    # Create copies of parents excluding the start and end cities
    modified_parent1 = parent1[1:-1]  # Exclude the first and last element
    modified_parent2 = parent2[1:-1]  # Exclude the first and last element

    child = modified_parent1[:]
    # Select a random city (gene) within the path, excluding start and end
    gene_index = random.randint(0, len(modified_parent1) - 1)
    gene = child[gene_index]
    
    # Find the position of the next city in the second parent
    next_index = (modified_parent2.index(gene) + 1) % len(modified_parent2)
    next_city = modified_parent2[next_index]
    
    next_index = child.index(next_city)

    # Perform the inversion of the subsequence between gene_index and next_index
    if gene_index < next_index:
        child[gene_index:next_index+1] = reversed(child[gene_index:next_index + 1])
    else:
        child[next_index:gene_index+1] = reversed(child[next_index:gene_index + 1])

    # Now we need to add the starting and ending cities from parent1[0] and parent1[-1]
    child.insert(0, parent1[0])
    child.append(parent1[0])
    
    return child
def inver_over_crossover_circular(parent1: List[int], parent2: List[int]) -> List[int]:
    # Create copies of parents without altering their structure
    modified_parent1 = parent1[:]  # Include all cities
    modified_parent2 = parent2[:]  # Include all cities

    child = modified_parent1[:]  # Start with a copy of the first parent
    
    # Select a random city (gene) within the path
    gene_index = random.randint(0, len(modified_parent1) - 1)
    gene = child[gene_index]

    # Find the position of the next city in the second parent
    next_index = (modified_parent2.index(gene) + 1) % len(modified_parent2)
    next_city = modified_parent2[next_index]

    next_index = child.index(next_city)

    # Perform the inversion of the subsequence between gene_index and next_index
    if gene_index < next_index:
        child[gene_index:next_index + 1] = reversed(child[gene_index:next_index + 1])
    else:
        child[next_index:gene_index + 1] = reversed(child[next_index:gene_index + 1])

    return child

# Crossover operator: Ordinal Crossover (OX)


    
def crossover(country_name:str ,data: Dict[str, Tuple[pd.DataFrame, np.ndarray]], parent1: List[int], parent2: List[int]) -> List[int]:
    # Set the first and last element as fixed (start/end city)
    cities, _ = data[country_name]
    num_cities = len(cities)
    start, end = sorted(random.sample(range(1, num_cities - 1), 2))
    child = [parent1[0]] + [None] * (num_cities - 2) + [parent1[0]]
    
    # Copy the selected segment from the first parent
    child[start:end] = parent1[start:end]
    
    # Complete the path with cities from the second parent, avoiding duplicates
    position = end
    for city in parent2:
        if city not in child:
            if position >= num_cities - 1:
                position = 1  # Restart from the second element to avoid first and last
            child[position] = city
            position += 1

    return child


# Crossover operator: Ordinal Crossover (OX)
def crossover_circular(parent1: List[int], parent2: List[int]) -> List[int]:
    # Set a segment of the path to be copied from the first parent
    start, end = sorted(random.sample(range(len(parent1)), 2))  # Include anche il primo e l'ultimo elemento
    child = [None] * len(parent1)  # Inizializza il figlio con valori None
    
    # Copy the selected segment from the first parent
    child[start:end] = parent1[start:end]
    
    # Complete the path with cities from the second parent, avoiding duplicates
    position = end
    for city in parent2:
        if city not in child:
            if position >= len(child):
                position = 0  # Restart from the beginning if we reach the end
            child[position] = city
            position += 1

    return child



## Mutation functions

In [23]:
# Mutation operator: Randomly swaps two cities
def mutate(individual: List[int], country_name: str, data: Dict[str, Tuple[pd.DataFrame, np.ndarray]] ) -> List[int]:
    cities, _ = data[country_name]
    if random.random() < MUTATION_RATE:
        # Select two cities to swap, excluding the starting and ending city
        i, j = random.sample(range(1, len(cities) - 1), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual



def inversion_mutation(individual: List[int]) -> List[int]:
    # Ensures the starting and ending city are fixed
    if random.random() < MUTATION_RATE:
        start_city = individual[0]
        end_city = individual[-1]
        
        # Select two random indices among the internal cities (1 to len(individual) - 2)
        idx1, idx2 = sorted(random.sample(range(1, len(individual) - 1), 2))
        
        # Reverse the sequence between idx1 and idx2
        individual[idx1:idx2 + 1] = reversed(individual[idx1:idx2 + 1])
        
        # Restore the starting and ending city
        individual[0] = start_city
        individual[-1] = end_city
    
    return individual

def inversion_mutation_circular(individual: List[int]) -> List[int]:
    # Allows inversion without fixing start and end
    if random.random() < MUTATION_RATE:
        # Select two random indices among all cities
        idx1, idx2 = sorted(random.sample(range(len(individual)), 2))
        
        # Reverse the sequence between idx1 and idx2
        individual[idx1:idx2 + 1] = reversed(individual[idx1:idx2 + 1])

    return individual




In [40]:

def genetic_algorithm(country_name: str, data: Dict[str, Tuple[pd.DataFrame, np.ndarray]], population: List[List[int]], max_generations: int, offspring_size: int, mutation_rate: float) -> List[int]:
    for generation in range(max_generations):
        # Offspring creation
        offspring = []
        for _ in range(offspring_size):
            parent1 = parent_selection(country_name, population, data)
            parent2 = parent_selection(country_name, population, data)
            child = inver_over_crossover( parent1, parent2)
            child = inversion_mutation(child) 
            offspring.append(child)
        # Fitness evaluation of the offspring
        population.extend(offspring)
        population.sort(key=lambda ind: fitness(country_name, ind, data), reverse=True)
        population = population[:len(population) - offspring_size]
        # Print every 100 generations
        if generation % 500 == 0:
            print(f"Generation {generation}, Best Path Cost: {path_cost(country_name, population[0], data):.2f} km")
    best_individual = population[0]
    # Print the best path found
    print("Best path found:", [data[country_name][0]['name'][i] for i in best_individual])
    print(f"Cost of best path: {path_cost(country_name, best_individual, data):.2f} km")
    return best_individual

In [39]:
def genetic_algorithm_circular(country_name: str, data: Dict[str, Tuple[pd.DataFrame, np.ndarray]], population: deque, max_generations: int, offspring_size: int) -> List[int]:
    cities, dist_matrix = data[country_name]

    for generation in range(max_generations):
        # Offspring creation
        offspring = []
        for _ in range(offspring_size):
            parent1 = parent_selection(country_name, population, data)
            parent2 = parent_selection(country_name, population, data)
            child = inver_over_crossover_circular( parent1, parent2)  # Usa la funzione di crossover aggiornata
            child = inversion_mutation_circular(child)     # Usa la funzione di mutazione aggiornata
            offspring.append(child)
        
        # Fitness evaluation of the offspring
        population.extend(offspring)
        population = deque(sorted(population, key=lambda ind: fitness_circular(country_name, ind, data ) , reverse=True)[:len(population) - offspring_size])  # Keeps the best ones for the initial size

        # Print every 100 generations
        if generation % 500 == 0:
            print(f"Generation {generation}, Best Path Cost: {path_cost_circular(country_name, population[0], data):.2f} km")

    # Final result
    best_individual = population[0]
    # Print the best path found
    print("Best path found:", [data[country_name][0]['name'][i] for i in best_individual])
    print(f"Cost of best path: {path_cost_circular(country_name, best_individual, data):.2f} km")
    return best_individual


# TourGene Algorithm

In [41]:
best_paths = []
for i, country in enumerate(data.keys()):
    print(f"Running genetic algorithm for {country}...")
    best_path = genetic_algorithm(country, data, population_all[i], MAX_GENERATIONS, OFFSPRING_SIZE, MUTATION_RATE)
    best_paths.append(best_path)

Running genetic algorithm for Italy...
Generation 0, Best Path Cost: 15074.77 km
Generation 500, Best Path Cost: 4606.45 km
Generation 1000, Best Path Cost: 4230.19 km
Generation 1500, Best Path Cost: 4182.68 km
Generation 2000, Best Path Cost: 4180.95 km
Generation 2500, Best Path Cost: 4176.28 km
Best path found: ['Ancona', 'Perugia', 'Terni', 'Rome', 'Latina', 'Pescara', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calabria', 'Syracuse', 'Catania', 'Palermo', 'Cagliari', 'Sassari', 'Leghorn', 'Florence', 'Prato', 'Bologna', 'Modena', "Reggio nell'Emilia", 'Parma', 'Piacenza', 'Genoa', 'Turin', 'Novara', 'Milan', 'Monza', 'Bergamo', 'Brescia', 'Verona', 'Trento', 'Bolzano', 'Trieste', 'Venice', 'Padua', 'Vicenza', 'Ferrara', 'Ravenna', 'Forlì', 'Rimini', 'Ancona']
Cost of best path: 4176.28 km
Running genetic algorithm for Russia...
Generation 0, Best Path Cost: 294344.40 km
Generation 500, Best Path Cost: 92710.36 km
Gene

# GreedGenesis Algorithm

In [37]:
best_paths_buffer = []
for i, country in enumerate(data.keys()):
    print(f"Running genetic algorithm for {country}...")
    best_path = genetic_algorithm_circular(country, data, population_all_buffer[i], MAX_GENERATIONS, OFFSPRING_SIZE)
    best_paths_buffer.append(best_path)

Running genetic algorithm for Italy...
Generation 0, Best Path Cost: 15069.19 km
Generation 500, Best Path Cost: 4420.12 km
Generation 1000, Best Path Cost: 4242.26 km
Generation 1500, Best Path Cost: 4175.99 km
Generation 2000, Best Path Cost: 4175.99 km
Generation 2500, Best Path Cost: 4175.99 km
Best path found: ['Trento', 'Bolzano', 'Vicenza', 'Padua', 'Venice', 'Trieste', 'Ravenna', 'Forlì', 'Rimini', 'Ancona', 'Perugia', 'Terni', 'Rome', 'Latina', 'Pescara', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calabria', 'Syracuse', 'Catania', 'Palermo', 'Cagliari', 'Sassari', 'Leghorn', 'Florence', 'Prato', 'Bologna', 'Ferrara', 'Modena', "Reggio nell'Emilia", 'Parma', 'Piacenza', 'Genoa', 'Turin', 'Novara', 'Milan', 'Monza', 'Bergamo', 'Brescia', 'Verona']
Cost of best path: 4175.99 km
Running genetic algorithm for Russia...
Generation 0, Best Path Cost: 303106.04 km
Generation 500, Best Path Cost: 74203.43 km
Generation 100

# RingPath Genetic Algorithm 

In [42]:
best_paths_greedy = []
for i, country in enumerate(data.keys()):
    print(f"Running genetic algorithm for {country}...")
    best_path = genetic_algorithm(country, data, population_all_greeedy[i], MAX_GENERATIONS, OFFSPRING_SIZE, MUTATION_RATE)
    best_paths_greedy.append(best_path)

Running genetic algorithm for Italy...
Generation 0, Best Path Cost: 4260.04 km
Generation 500, Best Path Cost: 4260.04 km
Generation 1000, Best Path Cost: 4260.04 km
Generation 1500, Best Path Cost: 4260.04 km
Generation 2000, Best Path Cost: 4260.04 km
Generation 2500, Best Path Cost: 4260.04 km
Best path found: ['Ancona', 'Pescara', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calabria', 'Syracuse', 'Catania', 'Palermo', 'Cagliari', 'Sassari', 'Latina', 'Rome', 'Terni', 'Perugia', 'Florence', 'Prato', 'Leghorn', 'Genoa', 'Turin', 'Novara', 'Milan', 'Monza', 'Bergamo', 'Trento', 'Bolzano', 'Trieste', 'Venice', 'Padua', 'Vicenza', 'Verona', 'Brescia', 'Piacenza', 'Parma', "Reggio nell'Emilia", 'Modena', 'Bologna', 'Ferrara', 'Ravenna', 'Forlì', 'Rimini', 'Ancona']
Cost of best path: 4260.04 km
Running genetic algorithm for Russia...
Generation 0, Best Path Cost: 35889.56 km
Generation 500, Best Path Cost: 35889.56 km
Genera